In [58]:
from typing import List, Dict
from tqdm import tqdm
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException

In [88]:
EXECUTABLE_PATH = "./resources/chromedriver.exe"

STOCK_MARKET_NEWS_URL = "https://www.investing.com/news/stock-market-news/"
POLITICS_NEWS_URL = "https://www.investing.com/news/politics/"
WORLD_NEWS_URL = "https://www.investing.com/news/world-news"

In [4]:
def get_driver(executable_path: str, page_load_timeout: int = 2):
    driver = webdriver.Chrome(executable_path)
    driver.set_page_load_timeout(page_load_timeout)
    
    return driver

In [5]:
def load_page(driver: webdriver.Chrome, link: str) -> None:
    try:
        driver.get(link)
    except TimeoutException:
        pass

In [47]:
def get_page_articles_links(driver: webdriver.Chrome, link: str) -> List[str]:
    load_page(driver, link)

    try:
        div_articles = driver.find_element(By.XPATH, "//div[contains(@class, 'largeTitle')]")
        refs = div_articles.find_elements(By.CLASS_NAME, "title")

        links = []
        for ref in refs:
            links.append(ref.get_attribute("href"))
        
        return links
    except:
        return []


def get_all_links(driver, link: str, num_pages: int = 3000) -> List[str]:
    links = []
    for page in tqdm(range(1, num_pages + 1)):
        current_link = link + str(page)
        
        links.extend(get_page_articles_links(driver, current_link))
        
    return links


In [81]:
def get_page_data(driver: webdriver.Chrome, link: str) -> Dict:
    load_page(driver, link)
    
    try:
        header_element = driver.find_element(By.XPATH, "//*[contains(@class, 'articleHeader')]")
        header = header_element.text
        
        body_element = driver.find_element(By.XPATH, "//div[contains(@class, 'articlePage')]")
        body = body_element.text
        
        content_element = driver.find_element(By.XPATH, "//div[contains(@class, 'contentSectionDetails')]")
        span_element = content_element.find_element(By.TAG_NAME, "span")
        date = span_element.text
        
        return {
            "header": header,
            "body": body,
            "date": date
        }
    except NoSuchElementException:
        return {}
    

def get_pages_data(driver: webdriver.Chrome, links: List[str]) -> List[Dict]:
    data = []
    for link in tqdm(links):
        page_data = get_page_data(driver, link)
        if len(page_data) > 0:
            data.append(page_data)
            
    return data


def get_news(driver: webdriver.Chrome, link: str, num_pages: int = 3000) -> List[Dict]:
    links = get_all_links(driver, link, num_pages)
    data = get_pages_data(driver, links)
    
    return data


def preprocess_data(data: List[Dict]) -> pd.DataFrame:
    df = pd.DataFrame(data)
    dates_parsed = df.date.str.findall("[^\(\)]+\sET")
    df.date = dates_parsed.apply(lambda date: pd.Timestamp(date[0], tz="US/Eastern").tz_convert(None))
    df = df.set_index("date")
    
    return df

In [89]:
driver = get_driver(EXECUTABLE_PATH)
data = get_news(driver, POLITICS_NEWS_URL, 100)

df = preprocess_data(data)
df.to_csv("politics_news.csv")

C:\Users\Виктор\AppData\Local\Temp\ipykernel_27200\1832538716.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path)
100%|██████████| 1718/1718 [1:06:01<00:00,  2.31s/it]
c:\ProgramData\Anaconda3\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname ET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [90]:
df

,header,body
date,,
2022-12-25 21:46:00,Migrants dropped outside VP Harris's home on b...,© Reuters. FILE PHOTO: U.S. Vice President Kam...
2022-12-24 19:30:00,Arizona judge dismisses election denier Kari L...,© Reuters. FILE PHOTO: Kari Lake speaks during...
2022-12-24 14:53:00,Factbox-Jan. 6 report adds to Trump's mounting...,© Reuters. FILE PHOTO: Former U.S. President D...
2022-12-24 14:52:00,Capitol riot panel's final report sets out cas...,© Reuters. FILE PHOTO: Members of the U.S. Hou...
2022-12-23 23:46:00,President Biden and first lady meet with patie...,© Reuters. U.S. President Joe Biden delivers a...
...,...,...
2022-02-15 11:27:00,Moderate Democrats fear party infighting could...,© Reuters. FILE PHOTO: Democratic congressiona...
2022-02-15 00:16:00,Manchin would not back Supreme Court confirmat...,© Reuters. FILE PHOTO: Chariman U.S. Senator J...
2022-02-15 00:11:00,Factbox-Ten key moments in Palin's defamation ...,"© Reuters. Sarah Palin, 2008 Republican vice p..."
